In [44]:
from langgraph_sdk import get_client

# Replace this with the URL of your own deployed graph
URL = "http://localhost:7777"
client = get_client(url=URL)

# Search all hosted graphs
assistants = await client.assistants.search()

In [244]:
import json

thread = await client.threads.create()
input_dict = {"request": 
              "i want to create a task about the team: approving leave requests, generate a report, and schedule a meeting with the team"}

async for chunk in client.runs.stream(
    thread["thread_id"],
    assistant_id="task_manager",
    input=input_dict,
    stream_mode="updates",):
    #    continue
    
    print(f"Receiving new event of type: {chunk.event}...")
    print(json.dumps(chunk.data, indent=4))
    print("\n\n")

Receiving new event of type: metadata...
{
    "run_id": "1efbbf14-c574-65b0-95ca-5ed36c4902f8",
    "attempt": 1
}



Receiving new event of type: updates...
{
    "__interrupt__": []
}





In [258]:
async for chunk in client.runs.stream(
    thread["thread_id"],
    assistant_id="task_manager",
    input=None,
    stream_mode="values",):
        #continue
    
    print(f"Receiving new event of type: {chunk.event}...")
    print(json.dumps(chunk.data, indent=4))
    print("\n\n")

Receiving new event of type: metadata...
{
    "run_id": "1efbbf1e-26d0-66b9-a023-7488cbfca6ae",
    "attempt": 1
}



Receiving new event of type: values...
{
    "request": "i want to approve leave requests, generate a report, and schedule a meeting with the team",
    "stored_procedures": {
        "procedures": [
            {
                "description": "review last bill received in gmail and pay it",
                "actions": [
                    {
                        "id": 0,
                        "type": "gmail",
                        "name": "read last 10 emails and find the bill",
                        "confirmed": true
                    },
                    {
                        "id": 1,
                        "type": "erp",
                        "name": "load bill into erp",
                        "confirmed": true
                    },
                    {
                        "id": 2,
                        "type": "bank",
                

In [259]:
current_state = await client.threads.get_state(thread['thread_id'])
current_state['next']

['executor']

In [260]:
current_state['tasks'][0]['interrupts']

[{'value': 'The task approve leave requests needs confirmation from the user',
  'resumable': False,
  'ns': None,
  'when': 'during'}]

In [253]:
await client.threads.update_state(
    thread['thread_id'], 
    {"request": "i want to approve leave requests, generate a report, and schedule a meeting with the team"}, 
    as_node="start")

{'checkpoint': {'thread_id': 'cec8b2e2-792a-424d-ac0b-1938c3f2260c',
  'checkpoint_ns': '',
  'checkpoint_id': '1efbbf1b-6f19-69ed-8007-e1e7eff470eb'},
 'configurable': {'thread_id': 'cec8b2e2-792a-424d-ac0b-1938c3f2260c',
  'checkpoint_ns': '',
  'checkpoint_id': '1efbbf1b-6f19-69ed-8007-e1e7eff470eb'},
 'checkpoint_id': '1efbbf1b-6f19-69ed-8007-e1e7eff470eb'}

In [219]:
async for chunk in client.runs.stream(
    #thread_id=current_state['checkpoint']['thread_id'],
    #checkpoint_id=current_state['checkpoint']['checkpoint_id'],
    thread_id=thread['thread_id'],
    assistant_id="task_manager",
    input=None,#{"human_confirmation_message": "yes"},
    stream_mode="updates",):
    #    continue
    
    print(f"Receiving new event of type: {chunk.event}...")
    print(json.dumps(chunk.data, indent=4))
    print("\n\n")

Receiving new event of type: metadata...
{
    "run_id": "1efbbd73-b0d8-62d5-bc64-0a815bab8423",
    "attempt": 1
}



Receiving new event of type: updates...
{
    "human_confirmation": null
}



Receiving new event of type: updates...
{
    "executor": {
        "actions_done": [
            {
                "id": 0,
                "type": "gmail",
                "name": "read last 10 emails and find the bill",
                "confirmed": true
            },
            {
                "id": 1,
                "type": "erp",
                "name": "load bill into erp",
                "confirmed": true
            },
            {
                "id": 2,
                "type": "bank",
                "name": "send payment to provider",
                "confirmed": false
            }
        ],
        "actions_pending": [],
        "human_feedback_message": "approve",
        "human_confirmation_message": "no"
    }
}



Receiving new event of type: updates...
{
    "human_

In [220]:
current_state = await client.threads.get_state(thread['thread_id'])
current_state['next']

['start']

In [221]:
current_state['tasks'][0]['interrupts']

[]

In [202]:
current_state

{'values': {'request': 'please pay the last bill in gmail',
  'stored_procedures': {'procedures': [{'description': 'review last bill received in gmail and pay it',
     'actions': [{'id': 0,
       'type': 'gmail',
       'name': 'read last 10 emails and find the bill',
       'confirmed': True},
      {'id': 1,
       'type': 'erp',
       'name': 'load bill into erp',
       'confirmed': True},
      {'id': 2,
       'type': 'bank',
       'name': 'send payment to provider',
       'confirmed': False}]},
    {'description': 'prepare salaries of employees and pay them',
     'actions': [{'id': 3,
       'type': 'google sheets',
       'name': 'calculate monthly hours per employee',
       'confirmed': True},
      {'id': 4,
       'type': 'erp',
       'name': 'load results into erp',
       'confirmed': True},
      {'id': 5,
       'type': 'bank',
       'name': 'send payment to employees',
       'confirmed': False}]},
    {'description': 'report on financial state of the company',